In [1]:
import numpy as np
import networkx as nx
import kmapper as km
import sklearn
import warnings
import matplotlib.pyplot as plt
import signal
import time
import random
from networkx.algorithms.similarity import graph_edit_distance

warnings.filterwarnings("ignore")

In [2]:
def select_k(spectrum, minimum_energy = 0.9):
    running_total = 0.0 
    
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

In [3]:
def calculate_similarity(graph1,graph2):
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)
    
    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2) #k are different between the two graphs, then use the smaller one.
    similarity = sum((laplacian1[:k] - laplacian2[:k])**2) #sum of the squared differences between the largest k eigenvalues
    return similarity


In [4]:
def extract_graph_features(graph):
    pr = nx.pagerank(graph,0.9)
    dc = nx.degree_centrality(graph)
    cc = nx.closeness_centrality(graph)
    bx = nx.betweenness_centrality(graph)
    c = nx.clustering(graph)
    
    #create list for each features
    pr_list =  [i for i in pr.values()]
    dc_list =  [i for i in dc.values()]
    cc_list =  [i for i in cc.values()]
    bx_list =  [i for i in bx.values()]
    c_list =  [i for i in c.values()]
    d_list = [val for (node, val) in graph.degree()]
    data = np.column_stack((pr_list,dc_list,cc_list,bx_list,c_list,d_list))
    return data

In [5]:
def TDA_transformation(data):
    Xfilt = data
    mapper = km.KeplerMapper()
    scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 1))
    Xfilt = scaler.fit_transform(Xfilt)
    lens = mapper.fit_transform(Xfilt, projection=sklearn.manifold.TSNE())
    cls = 2  # We use cls= 5

    graph = mapper.map(lens,Xfilt,clusterer=sklearn.cluster.KMeans(n_clusters=cls,random_state=1618033),
        cover=km.Cover(n_cubes=5, perc_overlap=0.3))
    return km.to_nx(graph)

In [6]:
def add_node_to_graph(graph,p):
    new_node = graph.number_of_nodes() + 1
    graph.add_node(new_node)
    existing_nodes = list(graph.nodes())[:-1]  # Exclude the new node
    for existing_node in existing_nodes:
        if random.random() < p:  
            graph.add_edge(new_node, existing_node)
    return graph
    

In [7]:
def remove_least_degree_node(graph):
    degrees = graph.degree()
    min_degree_node = min(degrees, key=lambda x: x[1])[0]
    # Remove the node with the minimum degree
    graph.remove_node(min_degree_node)
    return graph

In [8]:
def random_add_new_edge(graph):
    node_num = graph.number_of_nodes()
    node1 = random.randint(1,node_num)
    node2 = random.randint(1,node_num)
    while graph.has_edge(node1,node2) or node1 == node2 :
        node1 = random.randint(1,node_num)
        node2 = random.randint(1,node_num)
    graph.add_edge(node1,node2)
    return graph

In [9]:
def random_remove_edge(graph):
    random_edge = random.choice(list(graph.edges()))
    # Remove the randomly selected edge
    graph.remove_edge(*random_edge)
    return graph

In [10]:
def graph_generator(original_graph, node, edge,p):
    new_graph = original_graph.copy()
    if node < 0:
        for i in range(abs(node)):
            new_graph = remove_least_degree_node(new_graph)
    elif node > 0:
        for i in range(abs(node)):
            new_graph = add_node_to_graph(new_graph,p)
    
    if edge < 0:
        for i in range(abs(edge)):
            new_graph = random_remove_edge(new_graph)
    elif edge >0:
        for i in range(abs(edge)):
            new_graph = random_add_new_edge(new_graph)
    return new_graph

In [11]:
def calc_original_average_similarity_for_hop(graph, hop,p):
    counter = 0
    sum = 0
    for i in range(-hop,hop + 1):
        for j in range(-hop,hop + 1):
            if (i == -hop or i == hop or j == -hop or j == hop):
                neighbour = graph_generator(graph,j,i,p)
                score = calculate_similarity(graph, neighbour)
                sum += score
                counter += 1
    return sum/counter

In [12]:
def calc_TDA_average_similarity_for_hop(graph, hop,p):
    counter = 0
    sum = 0
    for i in range(-hop,hop + 1):
        for j in range(-hop,hop + 1):
            if (i == -hop or i == hop or j == -hop or j == hop):
                neighbour = graph_generator(graph,j,i,p)
                TDA_graph = TDA_transformation(extract_graph_features(graph))
                TDA_neighbour = TDA_transformation(extract_graph_features(neighbour))
                score = calculate_similarity(TDA_graph, TDA_neighbour)
                sum += score
                counter += 1
    return sum/counter

In [13]:
def calc_both_average_similarity_for_hop(graph, hop,p):
    counter = 0
    sum_original = 0
    sum_TDA = 0
    for i in range(-hop,hop + 1):
        for j in range(-hop,hop + 1):
            if (i == -hop or i == hop or j == -hop or j == hop):
                neighbour = graph_generator(graph,j,i,p)
                TDA_graph = TDA_transformation(extract_graph_features(graph))
                TDA_neighbour = TDA_transformation(extract_graph_features(neighbour))
                
                score_TDA = calculate_similarity(TDA_graph, TDA_neighbour)
                score_original = calculate_similarity(graph, neighbour)
                
                sum_TDA += score_TDA
                sum_original += score_original
                counter += 1
    return {"TDA": sum_TDA/counter,"original": sum_original/counter}

In [17]:
G = nx.erdos_renyi_graph(n=20,p=0.5)

In [16]:
result_original = calc_original_average_similarity_for_hop(G,2,0.9)
result_original

19.705066461042694

In [17]:
result_TDA = calc_TDA_average_similarity_for_hop(G,1,0.9)
result_TDA

3.625

In [124]:
result_TDA> result_original

False

In [194]:
G = nx.erdos_renyi_graph(n= 30,p=0.1) #n = 5, overlapping 0.1
dict = calc_both_average_similarity_for_hop(G,10,0.1)
TDA = dict['TDA']
original = dict['original']
print (original,TDA)

26.424260497985095 10.274605293822884


In [171]:
score = calc_TDA_average_similarity_for_hop(G,7,0.9)
score

62.28571428571428

In [14]:
def calc_hop():
    threshold = 100
    n = random.randint(30, 50)
    p = round(random.uniform(0, 100))
    reference_graph = nx.erdos_renyi_graph(n,p)
    
    TDA_average = 0
    original_average = 0
    
    TDA_hop = 0
    original_hop = 0
    while TDA_average <= threshold or original_average <= threshold:
        if(TDA_average <= threshold and original_average <= threshold):
            TDA_hop += 1
            original_hop += 1
            result = calc_both_average_similarity_for_hop(reference_graph,TDA_hop ,p)
            TDA_average = result['TDA']
            original_average = result['original']
            
        elif(TDA_average <= threshold):
            TDA_hop += 1
            TDA_average = calc_TDA_average_similarity_for_hop(reference_graph,TDA_hop,p)
        elif(original_average <= threshold):
            original_hop += 1
            original_average = calc_original_average_similarity_for_hop(reference_graph,original_hop,p)

    
    return {"Threshold": threshold, "n":n,"p":p,"last_TDA_average": TDA_average, "last_original_average": original_average,
           "TDA_hop": TDA_hop,"orginal_hop":original_hop}

In [ ]:
result = calc_hop()
result

In [15]:
def calc_hop_v1():
    threshold = 50
    n = random.randint(30,50)
    reach_max = False
    
    p = round(random.uniform(60, 100))/100
#     n = 30
#     p = 0.6
    reference_graph = nx.erdos_renyi_graph(n,p)
    
    TDA_average = 0
    original_average = 0
    
    TDA_hop = 0
    original_hop = 0
    while (TDA_average <= threshold or original_average <= threshold) and TDA_hop < 15:
        if(TDA_average <= threshold and original_average <= threshold):
            TDA_hop += 1
            original_hop += 1
            result = calc_both_average_similarity_for_hop(reference_graph,TDA_hop ,p)
            TDA_average = result['TDA']
            original_average = result['original']
            print(TDA_average,original_average)
        elif(TDA_average <= threshold):
            TDA_hop += 1
            TDA_average = calc_TDA_average_similarity_for_hop(reference_graph,TDA_hop,p)
            print(TDA_average)
        elif(original_average <= threshold):
            original_hop += 1
            original_average = calc_original_average_similarity_for_hop(reference_graph,original_hop,p)
            

    if(reach_max):
        return {"Threshold": threshold, "n":n,"p":p,"last_TDA_average": TDA_average, "last_original_average": original_average,
               "TDA_hop": TDA_hop + "+","orginal_hop":original_hop}
    else:
        return {"Threshold": threshold, "n":n,"p":p,"last_TDA_average": TDA_average, "last_original_average": original_average,
               "TDA_hop": TDA_hop,"orginal_hop":original_hop}
#     return {"TDA_hop":TDA_hop,"orginnal":original_hop}

In [19]:
result = calc_hop_v1()
result

25.193860090508146 103.17941832798756
25.814374849129177
23.59225980739336
20.56378693141322
25.23307418820466
29.64052264941301
22.662581091961815
29.770285349276335
27.833157828644755
28.88002958981512


{'Threshold': 50,
 'n': 48,
 'p': 0.63,
 'last_TDA_average': 28.88002958981512,
 'last_original_average': 103.17941832798756,
 'TDA_hop': 10,
 'orginal_hop': 1}

In [22]:
result = calc_hop_v1()
resul

33.95747837834349 1.984352623472304
16.69543439802717 8.257591325521911
23.325828771425382 32.003944793009865
33.468608513912784 59.0415088570316
21.67898955915578
25.429875965869684
30.93855982068564
25.073344062365262
24.524953563308376
21.213343072849156


NameError: name 'resul' is not defined

In [24]:
result

{'Threshold': 50,
 'n': 46,
 'p': 0.44,
 'last_TDA_average': 21.213343072849156,
 'last_original_average': 59.0415088570316,
 'TDA_hop': 10,
 'orginal_hop': 4}

In [26]:
result = calc_hop_v1()
resul

12.0 4.3569830386789254
20.25 13.299426233742658
25.151872092638172 22.42561983886885
20.989049392073984 29.429558895042405
20.632973309264578 46.930708689351846
20.45866147132654
20.00663025416645
25.244022286596586
23.01983158993419
21.524833201252175


NameError: name 'resul' is not defined

In [27]:
result

{'Threshold': 40,
 'n': 37,
 'p': 0.45,
 'last_TDA_average': 21.524833201252175,
 'last_original_average': 46.930708689351846,
 'TDA_hop': 10,
 'orginal_hop': 5}

In [29]:
result = calc_hop_v1()
result

12.566804130011953 120.90573354791978
14.73039321881345
13.109146678108738
14.120582269569542
15.467157287525378
18.134407515442813
25.245536677337103
18.95806016476702
20.903388914921383
20.609314575050757


{'Threshold': 35,
 'n': 38,
 'p': 0.79,
 'last_TDA_average': 20.609314575050757,
 'last_original_average': 120.90573354791978,
 'TDA_hop': 10,
 'orginal_hop': 1}

In [31]:
result = calc_hop_v1()
result

26.207478378343488 3.927960801589278
13.046524744255253 12.114885360773897
16.85247637020027 15.267114822646825
16.156333923293353 30.078317567891144
19.36723677732902
18.912836425745763
20.228785292163714
23.63583426054503
25.85344830252786
21.25726682628109


{'Threshold': 30,
 'n': 38,
 'p': 0.43,
 'last_TDA_average': 21.25726682628109,
 'last_original_average': 30.078317567891144,
 'TDA_hop': 10,
 'orginal_hop': 4}

In [38]:
result = calc_hop_v1()
result

40.916739700623296 513.47991885051
24.12393213510389
25.94781197854039
33.8795452549297
33.31390813090973
32.67761848897168
32.619262871888026
39.55643071866573
38.99047026603503
27.59012437032049


{'Threshold': 100,
 'n': 83,
 'p': 0.87,
 'last_TDA_average': 27.59012437032049,
 'last_original_average': 513.47991885051,
 'TDA_hop': 10,
 'orginal_hop': 1}

In [21]:
result = calc_hop_v1()
result

26.376235651622558 27.4192500778991
69.17230412869012 1222.264599049612


{'Threshold': 50,
 'n': 80,
 'p': 0.89,
 'last_TDA_average': 69.17230412869012,
 'last_original_average': 1222.264599049612,
 'TDA_hop': 2,
 'orginal_hop': 2}

In [17]:
resul_1 = calc_hop_v1()
resul_2 = calc_hop_v1()
resul_3 = calc_hop_v1()
resul_4 = calc_hop_v1()
resul_5 = calc_hop_v1()


46.15105621630013 11.015439785860323
37.00546827335259 130.4601520451427
31.036524448794733
30.04461939916994
49.14715171055107
39.25117437298724
38.636599481201564
44.57898576389495
40.64931810489352
44.45846252301246
26.13301325849323 5.114000735734586
42.01661158405077 10.468802752718963
27.079544883834245 31.7352764377621
50.756795278468445 63.460675004221386
31.43039807830629 83.23096308168236
36.296533057504824 137.70754241067064
41.4754168842221
48.30208624746061
32.00189829605429
41.369250961069
59.94918355800154 18.446056139691798
55.14740750230365 242.59063078054822
55.56177686392231
50.96186876395127
46.597006797341805
44.481114531249375
60.75804134636349
42.61434493687747
60.48204012606319
61.73353893401653
73.1836715285251 3.872730673751498
52.64590681048319 7.640418237187698
34.39868829236225 27.19138894457161
43.755002797146595 25.65954813346606
42.98364141897164 54.504411998104295
48.47038042600815 83.54283795416858
43.310839981643745 104.3634304514952
43.15051394322556

In [22]:
print(resul_1)
print(resul_2)
print(resul_3)
print(resul_4)
print(resul_5)

{'Threshold': 100, 'n': 99, 'p': 0.56, 'last_TDA_average': 44.45846252301246, 'last_original_average': 130.4601520451427, 'TDA_hop': 10, 'orginal_hop': 2}
{'Threshold': 100, 'n': 86, 'p': 0.36, 'last_TDA_average': 41.369250961069, 'last_original_average': 137.70754241067064, 'TDA_hop': 10, 'orginal_hop': 6}
{'Threshold': 100, 'n': 83, 'p': 0.78, 'last_TDA_average': 61.73353893401653, 'last_original_average': 242.59063078054822, 'TDA_hop': 10, 'orginal_hop': 2}
{'Threshold': 100, 'n': 91, 'p': 0.3, 'last_TDA_average': 42.046974413350405, 'last_original_average': 104.3634304514952, 'TDA_hop': 10, 'orginal_hop': 7}
{'Threshold': 100, 'n': 89, 'p': 0.64, 'last_TDA_average': 54.28764792817866, 'last_original_average': 244.2480144239988, 'TDA_hop': 10, 'orginal_hop': 1}


In [24]:
resul_6 = calc_hop_v1()
resul_7 = calc_hop_v1()
resul_8 = calc_hop_v1()
resul_9 = calc_hop_v1()
resul_10 = calc_hop_v1()

95.61182435689912 36.42469473349315
30.414330475776246 17.058330631893657
39.85612880980464 217.74987097658163
52.87534030128878
48.20191208279964
57.427815026009355
40.62083677301164
40.07483205931747
53.46429574503318
48.391137253138815
52.040411702282235
23.564394503526177 16.574395614986134
35.812834767972014 356.5554509212506
48.15119840885348
40.45557517349697
31.24795188415705
35.679503223289046
38.145953162592356
37.51185644784383
36.38532110153793
40.00022079906263
48.61117008459621 443.83664887295555
47.133481796945304
42.12772717368177
45.3758163182611
55.80230750036374
57.316619685846916
55.14932032879888
60.21365279097193
24.465115478613978 11.066914894037566
39.71132991622363 35.83812476488064
26.20352650945488 147.41254426643715
41.13649098035807
30.99913170960952
40.536129568340655
34.6839342577927
33.30852284852589
38.3456694073408
40.9479046062329


In [25]:
print(resul_6)
print(resul_7)
print(resul_8)
print(resul_9)
print(resul_10)


{'Threshold': 60, 'n': 80, 'p': 0.91, 'last_TDA_average': 95.61182435689912, 'last_original_average': 929.2582593148135, 'TDA_hop': 1, 'orginal_hop': 2}
{'Threshold': 60, 'n': 95, 'p': 0.68, 'last_TDA_average': 52.040411702282235, 'last_original_average': 217.74987097658163, 'TDA_hop': 10, 'orginal_hop': 2}
{'Threshold': 60, 'n': 89, 'p': 0.72, 'last_TDA_average': 40.00022079906263, 'last_original_average': 356.5554509212506, 'TDA_hop': 10, 'orginal_hop': 2}
{'Threshold': 60, 'n': 83, 'p': 0.83, 'last_TDA_average': 60.21365279097193, 'last_original_average': 443.83664887295555, 'TDA_hop': 8, 'orginal_hop': 1}
{'Threshold': 60, 'n': 93, 'p': 0.57, 'last_TDA_average': 40.9479046062329, 'last_original_average': 147.41254426643715, 'TDA_hop': 10, 'orginal_hop': 3}


In [27]:
resul_11 = calc_hop_v1()
resul_12 = calc_hop_v1()
resul_13 = calc_hop_v1()
resul_14 = calc_hop_v1()
resul_15 = calc_hop_v1()

29.523839463580885 26.785883248338894
40.05176410796843 72.27871977071324
21.427231699341196
36.93674137761426
48.282044571436074
41.26416035977493
40.841548844283444
44.7614671607197
34.345009394514015
45.49581839206694
45.81936233858943 698.2433243597509
39.18081817573012
40.81734833189379
49.3382924045815
42.02962210426891
48.432591290174685
43.61419830701925
59.17936664482337
29.461021375999948 5.636066999351328
39.960520032608116 105.48878242816266
34.102890796261455
51.14490810019318
19.14747190559294 2.482150518742524
37.4312324761461 20.703021805312556
24.552957023719543 32.318926309257904
40.33691586062894 62.39845429375215
27.945767473188805
24.55300462595436
33.019395196191304
34.33074130265186
32.20993700569283
36.47986260352272
30.879219022048492 3.291601372867979
63.00091150427142 36.61122784286264


In [28]:
print(resul_11)
print(resul_12)
print(resul_13)
print(resul_14)
print(resul_15)

{'Threshold': 50, 'n': 93, 'p': 0.72, 'last_TDA_average': 45.49581839206694, 'last_original_average': 72.27871977071324, 'TDA_hop': 10, 'orginal_hop': 2}
{'Threshold': 50, 'n': 98, 'p': 0.88, 'last_TDA_average': 59.17936664482337, 'last_original_average': 698.2433243597509, 'TDA_hop': 8, 'orginal_hop': 1}
{'Threshold': 50, 'n': 90, 'p': 0.41, 'last_TDA_average': 51.14490810019318, 'last_original_average': 105.48878242816266, 'TDA_hop': 4, 'orginal_hop': 2}
{'Threshold': 50, 'n': 85, 'p': 0.31, 'last_TDA_average': 36.47986260352272, 'last_original_average': 62.39845429375215, 'TDA_hop': 10, 'orginal_hop': 4}
{'Threshold': 50, 'n': 80, 'p': 0.41, 'last_TDA_average': 63.00091150427142, 'last_original_average': 79.33839768351557, 'TDA_hop': 2, 'orginal_hop': 3}


In [16]:
resul_16 = calc_hop_v1()
resul_17 = calc_hop_v1()
resul_18 = calc_hop_v1()
resul_19 = calc_hop_v1()
resul_20 = calc_hop_v1()

31.239126633040847 47.65279426904322
27.574417669423003 40.793619707920605
38.701012599021446 138.2301401081623
49.984477422088105
33.803019140498954
41.9263070399669
30.875850411306256
34.07409245869962
41.91182029879078
46.343562239267015
46.203569693636126
42.999523486514384
35.59580514169789
50.20687395651617
30.56460454324271 23.034255399623724
47.17919686770146 45.54201537328809
51.316289956522894 47.141799200342035
55.90489978959863 121.70329471644074
23.379721115416764 257.7923623902407
56.764394839907695
78.904554255636 202.96078643762687


In [17]:
print(resul_16)
print(resul_17)
print(resul_18)
print(resul_19)
print(resul_20)

{'Threshold': 50, 'n': 32, 'p': 0.76, 'last_TDA_average': 50.20687395651617, 'last_original_average': 138.2301401081623, 'TDA_hop': 14, 'orginal_hop': 3}
{'Threshold': 50, 'n': 33, 'p': 0.66, 'last_TDA_average': 51.316289956522894, 'last_original_average': 91.37541186087155, 'TDA_hop': 3, 'orginal_hop': 4}
{'Threshold': 50, 'n': 49, 'p': 0.8, 'last_TDA_average': 55.90489978959863, 'last_original_average': 121.70329471644074, 'TDA_hop': 1, 'orginal_hop': 1}
{'Threshold': 50, 'n': 49, 'p': 0.83, 'last_TDA_average': 56.764394839907695, 'last_original_average': 257.7923623902407, 'TDA_hop': 2, 'orginal_hop': 1}
{'Threshold': 50, 'n': 31, 'p': 0.99, 'last_TDA_average': 78.904554255636, 'last_original_average': 202.96078643762687, 'TDA_hop': 1, 'orginal_hop': 1}
